<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Homework/4_Classification/PH_5_Bayes_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 
Apply our two new models (multinomial naive Bayes and a neural network) to the 20-Newsgroups dataset.

In [1]:
# imports
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load datasets

In [3]:
#Load stemmed data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_stemmed.pkl', 'rb') as f:
    data_stemmed_list = pickle.load(f)

stemmed_df = pd.DataFrame(data_stemmed_list, columns=["stemmed_text"])

In [4]:
#Load lemmatized data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_lemma.pkl', 'rb') as f:
    data_lemma_list = pickle.load(f)

lemma_df = pd.DataFrame(data_lemma_list, columns=["stemmed_text"])

In [5]:
# load y data
y_df = pd.read_json("https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json")[["target","target_names"]]
y_df.head()

,target,target_names
0,7,rec.autos
1,4,comp.sys.mac.hardware
2,4,comp.sys.mac.hardware
3,1,comp.graphics
4,14,sci.space


In [6]:
stemmed_y_df = pd.concat([stemmed_df, y_df], axis=1)
lemma_y_df = pd.concat([lemma_df, y_df], axis=1)

In [7]:
lemma_y_df.columns

Index(['stemmed_text', 'target', 'target_names'], dtype='object')

In [8]:
# train test split
X_stem = stemmed_y_df[["stemmed_text"]]
y_stem = stemmed_y_df['target']

X_s_train, X_s_test, y_s_train, y_s_test = train_test_split(X_stem, y_stem, test_size=0.1)

# train test split
X_lem = lemma_y_df[["stemmed_text"]]
y_lem = lemma_y_df['target']

X_l_train, X_l_test, y_l_train, y_l_test = train_test_split(X_lem, y_lem, test_size=0.1)

## Multinomial Naive Bayes

In [9]:
# Instantiate Vectorizers 

#Stemmed
abs_vec_s = CountVectorizer(max_df=0.95, min_df=0.01)
rel_vec_s = TfidfVectorizer(max_df=0.95, min_df=0.01, use_idf=False, norm='l1') # use-idf: Enable inverse-document-frequency reweighting; norm: Each output row will have unit norm (l1 = Sum of absolute values of vector elements is 1.)
idf_vec_s = TfidfVectorizer(max_df=0.95, min_df=0.01, smooth_idf=True) # smooth_idf: Smooth idf weights by adding one to document frequencies (Prevents zero divisions) -> not regularized

#Lemma
abs_vec_l = CountVectorizer(max_df=0.95, min_df=0.01)
rel_vec_l = TfidfVectorizer(max_df=0.95, min_df=0.01, use_idf=False, norm='l1') # use-idf: Enable inverse-document-frequency reweighting; norm: Each output row will have unit norm (l1 = Sum of absolute values of vector elements is 1.)
idf_vec_l = TfidfVectorizer(max_df=0.95, min_df=0.01, smooth_idf=True) # smooth_idf: Smooth idf weights by adding one to document frequencies (Prevents zero divisions) -> not regularized

### Abs. Freq. 

#### Stemmed

In [10]:
X_train_abs_stem = abs_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_abs_stem = abs_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [11]:
clf_s = MultinomialNB()
clf_s.fit(X_train_abs_stem, y_s_train)

MultinomialNB()

In [12]:
y_pred_s = clf_s.predict(X_test_abs_stem)
print('Train accuracy: ', clf_s.score(X_train_abs_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.8457081123551365
Test accuracy:  0.7800353356890459


#### Lemmatized

In [13]:
X_train_abs_lem = abs_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_abs_lem = abs_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [14]:
clf_l = MultinomialNB()
clf_l.fit(X_train_abs_lem, y_l_train)

MultinomialNB()

In [15]:
y_pred_l = clf_l.predict(X_test_abs_lem)
print('Train accuracy: ', clf_l.score(X_train_abs_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.8405028481634257
Test accuracy:  0.7729681978798587


### Rel. Freq. 

#### Stemmed

In [16]:
X_train_rel_stem = rel_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_rel_stem = rel_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [17]:
clf_s = MultinomialNB()
clf_s.fit(X_train_rel_stem, y_s_train)

MultinomialNB()

In [18]:
y_pred_s = clf_s.predict(X_test_rel_stem)
print('Train accuracy: ', clf_s.score(X_train_rel_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.7485759182871734
Test accuracy:  0.6855123674911661


#### Lemmatized

In [19]:
X_train_rel_lem = rel_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_rel_lem = rel_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [20]:
clf_l = MultinomialNB()
clf_l.fit(X_train_rel_lem, y_l_train)

MultinomialNB()

In [21]:
y_pred_l = clf_l.predict(X_test_rel_lem)
print('Train accuracy: ', clf_l.score(X_train_rel_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.7253977607542722
Test accuracy:  0.6651943462897526


### TF-Idf

#### Stemmed

In [22]:
X_train_idf_stem = idf_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_idf_stem = idf_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [23]:
clf_s = MultinomialNB()
clf_s.fit(X_train_idf_stem, y_s_train)

MultinomialNB()

In [24]:
y_pred_s = clf_s.predict(X_test_idf_stem)
print('Train accuracy: ', clf_s.score(X_train_idf_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.8613239049302691
Test accuracy:  0.7853356890459364


#### Lemmatized

In [25]:
X_train_idf_lem = idf_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_idf_lem = idf_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [26]:
clf_l = MultinomialNB()
clf_l.fit(X_train_idf_lem, y_l_train)

MultinomialNB()

In [27]:
y_pred_l = clf_l.predict(X_test_idf_lem)
print('Train accuracy: ', clf_l.score(X_train_idf_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.8508151640149283
Test accuracy:  0.7932862190812721


## Neural Networks

In [67]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

In [68]:
# no of classes in dataset
y_df.target.nunique()

20

In [69]:
from tensorflow.keras.utils import to_categorical
y_train_l_cat = to_categorical(y_l_train)
y_test_l_cat = to_categorical(y_l_test)

In [70]:
y_train_s_cat = to_categorical(y_s_train)
y_test_s_cat = to_categorical(y_s_test)

In [75]:
#Early stopping
es = EarlyStopping(patience=20, restore_best_weights=True)

### Abs. Freq. 

#### Stemmed

In [71]:
#input dim
X_train_abs_stem.shape[1]

1641

In [72]:
# Classification with 20 classes
model_s = Sequential()
model_s.add(layers.Dense(30, activation='relu', input_dim=X_train_abs_stem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model_s.add(layers.Dense(20, activation='softmax'))

In [73]:
model_s.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [74]:
model_s.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 30)                49260     
                                                                 
 dense_9 (Dense)             (None, 20)                620       
                                                                 
Total params: 49,880
Trainable params: 49,880
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Fit
model_s.fit(np.asarray(X_train_abs_stem).astype("float32"), y_train_s_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.1877 - accuracy: 0.4879 - val_loss: 1.4452 - val_accuracy: 0.7103
Epoch 2/100
223/223 [==============================] - 1s 3ms/step - loss: 1.0308 - accuracy: 0.8096 - val_loss: 1.0129 - val_accuracy: 0.7614
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 0.6691 - accuracy: 0.8727 - val_loss: 0.8547 - val_accuracy: 0.7781
Epoch 4/100
223/223 [==============================] - 1s 3ms/step - loss: 0.4860 - accuracy: 0.9050 - val_loss: 0.7950 - val_accuracy: 0.7866
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 0.3723 - accuracy: 0.9310 - val_loss: 0.7648 - val_accuracy: 0.7948
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 0.2932 - accuracy: 0.9498 - val_loss: 0.7819 - val_accuracy: 0.7899
Epoch 7/100
223/223 [==============================] - 1s 3ms/step - loss: 0.2352 - accuracy: 0.9620 - val_loss: 0.7483 - val_accuracy: 0.7951

In [86]:
results = model_s.evaluate(np.asarray(X_test_abs_stem).astype("float32"), y_test_s_cat, verbose=0)
print(results[1]) #accuracy

0.7985865473747253


#### Lemmatized

In [78]:
#input dim
X_train_abs_lem.shape

(10182, 1575)

In [79]:
# Classification with 20 classes
model = Sequential()
model.add(layers.Dense(30, activation='relu', input_dim=X_train_abs_lem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [80]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [81]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 30)                47280     
                                                                 
 dense_11 (Dense)            (None, 20)                620       
                                                                 
Total params: 47,900
Trainable params: 47,900
Non-trainable params: 0
_________________________________________________________________


In [82]:
#Early stopping
es = EarlyStopping(patience=20, restore_best_weights=True)

In [83]:
# Fit
model.fit(np.asarray(X_train_abs_lem).astype("float32"), y_train_l_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.2346 - accuracy: 0.4663 - val_loss: 1.5206 - val_accuracy: 0.7074
Epoch 2/100
223/223 [==============================] - 1s 3ms/step - loss: 1.1029 - accuracy: 0.7883 - val_loss: 1.0782 - val_accuracy: 0.7722
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 0.7269 - accuracy: 0.8597 - val_loss: 0.9308 - val_accuracy: 0.7820
Epoch 4/100
223/223 [==============================] - 1s 3ms/step - loss: 0.5424 - accuracy: 0.8946 - val_loss: 0.8709 - val_accuracy: 0.7957
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 0.4223 - accuracy: 0.9192 - val_loss: 0.8462 - val_accuracy: 0.7951
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 0.3359 - accuracy: 0.9413 - val_loss: 0.8386 - val_accuracy: 0.8013
Epoch 7/100
223/223 [==============================] - 1s 3ms/step - loss: 0.2715 - accuracy: 0.9557 - val_loss: 0.8725 - val_accuracy: 0.7921

In [84]:
results = model.evaluate(np.asarray(X_test_abs_lem).astype("float32"), y_test_l_cat, verbose=0)
print(results[1]) #accuracy

0.804770290851593


### Rel. Freq. 

#### Stemmed

In [88]:
#input dim
X_train_rel_stem.shape[1]

1641

In [89]:
# Classification with 20 classes
model_s = Sequential()
model_s.add(layers.Dense(30, activation='relu', input_dim=X_train_rel_stem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model_s.add(layers.Dense(20, activation='softmax'))

In [90]:
model_s.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [91]:
model_s.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 30)                49260     
                                                                 
 dense_13 (Dense)            (None, 20)                620       
                                                                 
Total params: 49,880
Trainable params: 49,880
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Fit
model_s.fit(np.asarray(X_train_rel_stem).astype("float32"), y_train_s_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.9565 - accuracy: 0.2078 - val_loss: 2.8909 - val_accuracy: 0.4468
Epoch 2/100
223/223 [==============================] - 1s 4ms/step - loss: 2.7538 - accuracy: 0.5786 - val_loss: 2.6189 - val_accuracy: 0.5741
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 2.3936 - accuracy: 0.6607 - val_loss: 2.2531 - val_accuracy: 0.6354
Epoch 4/100
223/223 [==============================] - 1s 3ms/step - loss: 2.0060 - accuracy: 0.6972 - val_loss: 1.9263 - val_accuracy: 0.6661
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 1.6852 - accuracy: 0.7383 - val_loss: 1.6737 - val_accuracy: 0.6854
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 1.4382 - accuracy: 0.7636 - val_loss: 1.4804 - val_accuracy: 0.7119
Epoch 7/100
223/223 [==============================] - 1s 3ms/step - loss: 1.2481 - accuracy: 0.7835 - val_loss: 1.3384 - val_accuracy: 0.7231

In [93]:
results = model_s.evaluate(np.asarray(X_test_rel_stem).astype("float32"), y_test_s_cat, verbose=0)
print(results[1]) #accuracy

0.8038869500160217


#### Lemmatized

In [94]:
#input dim
X_train_rel_lem.shape

(10182, 1575)

In [95]:
# Classification with 20 classes
model = Sequential()
model.add(layers.Dense(30, activation='relu', input_dim=X_train_rel_lem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [96]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [97]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 30)                47280     
                                                                 
 dense_15 (Dense)            (None, 20)                620       
                                                                 
Total params: 47,900
Trainable params: 47,900
Non-trainable params: 0
_________________________________________________________________


In [98]:
#Early stopping
es = EarlyStopping(patience=20, restore_best_weights=True)

In [99]:
# Fit
model.fit(np.asarray(X_train_rel_lem).astype("float32"), y_train_l_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.9549 - accuracy: 0.2718 - val_loss: 2.8881 - val_accuracy: 0.5080
Epoch 2/100
223/223 [==============================] - 1s 3ms/step - loss: 2.7593 - accuracy: 0.5378 - val_loss: 2.6211 - val_accuracy: 0.5489
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 2.4231 - accuracy: 0.6231 - val_loss: 2.2698 - val_accuracy: 0.6079
Epoch 4/100
223/223 [==============================] - 1s 5ms/step - loss: 2.0584 - accuracy: 0.6694 - val_loss: 1.9461 - val_accuracy: 0.6576
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 1.7469 - accuracy: 0.7104 - val_loss: 1.6883 - val_accuracy: 0.6884
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 1.5016 - accuracy: 0.7364 - val_loss: 1.4926 - val_accuracy: 0.7126
Epoch 7/100
223/223 [==============================] - 1s 3ms/step - loss: 1.3107 - accuracy: 0.7647 - val_loss: 1.3435 - val_accuracy: 0.7349

In [100]:
results = model.evaluate(np.asarray(X_test_rel_lem).astype("float32"), y_test_l_cat, verbose=0)
print(results[1]) #accuracy

0.804770290851593


### Tf-Idf

#### Stemmed

In [102]:
#input dim
X_train_idf_stem.shape[1]

1641

In [103]:
# Classification with 20 classes
model_s = Sequential()
model_s.add(layers.Dense(30, activation='relu', input_dim=X_train_idf_stem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model_s.add(layers.Dense(20, activation='softmax'))

In [104]:
model_s.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [105]:
model_s.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 30)                49260     
                                                                 
 dense_17 (Dense)            (None, 20)                620       
                                                                 
Total params: 49,880
Trainable params: 49,880
Non-trainable params: 0
_________________________________________________________________


In [106]:
# Fit
model_s.fit(np.asarray(X_train_idf_stem).astype("float32"), y_train_s_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.7517 - accuracy: 0.3804 - val_loss: 2.3809 - val_accuracy: 0.6098
Epoch 2/100
223/223 [==============================] - 1s 5ms/step - loss: 1.8628 - accuracy: 0.7258 - val_loss: 1.5603 - val_accuracy: 0.7296
Epoch 3/100
223/223 [==============================] - 1s 6ms/step - loss: 1.1937 - accuracy: 0.8109 - val_loss: 1.1444 - val_accuracy: 0.7620
Epoch 4/100
223/223 [==============================] - 1s 5ms/step - loss: 0.8566 - accuracy: 0.8436 - val_loss: 0.9442 - val_accuracy: 0.7777
Epoch 5/100
223/223 [==============================] - 1s 6ms/step - loss: 0.6698 - accuracy: 0.8682 - val_loss: 0.8304 - val_accuracy: 0.7902
Epoch 6/100
223/223 [==============================] - 1s 5ms/step - loss: 0.5485 - accuracy: 0.8852 - val_loss: 0.7637 - val_accuracy: 0.8003
Epoch 7/100
223/223 [==============================] - 1s 6ms/step - loss: 0.4614 - accuracy: 0.9023 - val_loss: 0.7190 - val_accuracy: 0.8072

In [107]:
results = model_s.evaluate(np.asarray(X_test_idf_stem).astype("float32"), y_test_s_cat, verbose=0)
print(results[1]) #accuracy

0.8074204921722412


#### Lemmatized

In [108]:
#input dim
X_train_idf_lem.shape

(10182, 1575)

In [109]:
# Classification with 20 classes
model = Sequential()
model.add(layers.Dense(30, activation='relu', input_dim=X_train_idf_lem.shape[1]))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [110]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [111]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 30)                47280     
                                                                 
 dense_19 (Dense)            (None, 20)                620       
                                                                 
Total params: 47,900
Trainable params: 47,900
Non-trainable params: 0
_________________________________________________________________


In [112]:
#Early stopping
es = EarlyStopping(patience=20, restore_best_weights=True)

In [113]:
# Fit
model.fit(np.asarray(X_train_idf_lem).astype("float32"), y_train_l_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 1s 4ms/step - loss: 2.7725 - accuracy: 0.4149 - val_loss: 2.3947 - val_accuracy: 0.6026
Epoch 2/100
223/223 [==============================] - 1s 3ms/step - loss: 1.9118 - accuracy: 0.7161 - val_loss: 1.5453 - val_accuracy: 0.7332
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 1.2248 - accuracy: 0.7950 - val_loss: 1.1252 - val_accuracy: 0.7715
Epoch 4/100
223/223 [==============================] - 1s 3ms/step - loss: 0.8867 - accuracy: 0.8311 - val_loss: 0.9286 - val_accuracy: 0.7869
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 0.7015 - accuracy: 0.8532 - val_loss: 0.8213 - val_accuracy: 0.7954
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 0.5807 - accuracy: 0.8758 - val_loss: 0.7556 - val_accuracy: 0.8020
Epoch 7/100
223/223 [==============================] - 1s 3ms/step - loss: 0.4925 - accuracy: 0.8929 - val_loss: 0.7158 - val_accuracy: 0.8082

In [114]:
results = model.evaluate(np.asarray(X_test_idf_lem).astype("float32"), y_test_l_cat, verbose=0)
print(results[1]) #accuracy

0.8074204921722412
